In [1]:
!pip install spacy yake transformers torch
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:

# Notes:
#  - This implementation uses rule-based extraction + spaCy for tokenization.
#  - Optional transformers summarizer/classifier placeholders are included but not required.

import re
import json
from collections import defaultdict

try:
    import spacy
except Exception as e:
    raise ImportError("Please install spaCy: pip install spacy") from e

try:
    import yake
except Exception as e:
    raise ImportError("Please install yake: pip install yake") from e



nlp = spacy.load("en_core_web_sm")

def normalize_transcript(text: str) -> str:
    text = text.strip()
    text = re.sub(r'\r\n', '\n', text)
    text = re.sub(r'\s+\n', '\n', text)
    text = re.sub(r'\n\s+', '\n', text)
    return text

def split_speaker_segments(text: str):
    """
    Splits transcript into list of {"speaker": ..., "text": ...}.
    Recognizes 'Physician', 'Doctor', 'Patient', 'Ms. Jones' etc. Falls back to previous speaker for continuations.
    """
    segments = []
    for raw_line in text.splitlines():
        line = raw_line.strip()
        if not line:
            continue
        m = re.match(r'^(Physician|Doctor|Patient|Ms\.?\s+Jones|Mr\.|Mrs\.)[:\-]?\s*(.*)$', line, flags=re.I)
        if m:
            speaker = m.group(1).strip()

            if re.search(r'^(physician|doctor)$', speaker, flags=re.I):
                speaker_tag = "Physician"
            elif re.search(r'patient', speaker, flags=re.I):
                speaker_tag = "Patient"
            else:
                speaker_tag = "Patient" if "Ms" in speaker or "Mr" in speaker or "Mrs" in speaker else speaker
            utterance = m.group(2).strip()
            segments.append({"speaker": speaker_tag, "text": utterance})
        else:

            if segments:
                segments[-1]["text"] = segments[-1]["text"].rstrip() + " " + line
            else:
                segments.append({"speaker": "Unknown", "text": line})
    return segments

# ---------- Rule-based medical entity extraction ----------
SYMPTOM_KEYWORDS = [
    "pain", "ache", "stiffness", "head", "neck", "back", "nausea",
    "dizziness", "anxiety", "sleep", "sleeping", "trouble sleeping",
    "backache", "headache", "tenderness", "stiff", "stiffness"
]
TREATMENT_KEYWORDS = [
    "physiotherapy", "physio", "painkiller", "analgesic", "x-ray",
    "xray", "surgery", "therapy", "session", "sessions", "advice"
]
DIAGNOSIS_KEYWORDS = [
    "whiplash", "fracture", "concussion", "sprain", "strain"
]

def rule_extract_entities(text: str):
    text_l = text.lower()
    symptoms = set()
    treatments = set()
    diagnosis = set()

    # Symptoms
    for kw in SYMPTOM_KEYWORDS:
        if re.search(r'\b' + re.escape(kw) + r'\b', text_l):
            # normalize some tokens
            if kw in ("sleep", "sleeping", "trouble sleeping"):
                symptoms.add("sleep disturbance")
            elif kw in ("backache",):
                symptoms.add("back pain")
            elif kw in ("head", "headache"):
                symptoms.add("head impact")
            elif kw in ("neck",):
                symptoms.add("neck pain")
            else:
                # generic mapping
                if kw in ("pain", "ache"):
                    # try to capture body part near the word
                    ctx = _extract_context_around_word(text_l, kw, window=40)
                    found = _extract_body_part_from_context(ctx)
                    if found:
                        symptoms.add(found + " pain")
                    else:
                        symptoms.add("pain")
                else:
                    symptoms.add(kw)

    # Diagnosis
    for kw in DIAGNOSIS_KEYWORDS:
        if re.search(r'\b' + re.escape(kw) + r'\b', text_l):
            diagnosis.add(kw)

    # Treatment: look for sessions and specific treatments
    for kw in TREATMENT_KEYWORDS:
        if re.search(r'\b' + re.escape(kw) + r'\b', text_l):
            treatments.add(kw)

    # numeric physiotherapy pattern
    m = re.search(r'(\d+)\s+(physiotherapy|sessions|session|physio sessions|physio)', text_l)
    if m:
        treatments.add(f"{m.group(1)} physiotherapy sessions")

    # painkillers / analgesic patterns
    if re.search(r'\b(painkillers|pain killers|analgesics|analgesic)\b', text_l):
        treatments.add("analgesics / painkillers")

    # head impact detection
    if re.search(r'(hit my head|hit my head on|head on the steering|hit my head)', text_l):
        symptoms.add("head impact")

    # seatbelt info (safety)
    if re.search(r'\bseatbelt\b', text_l):
        # we won't add as symptom but keep track in metadata if needed
        pass

    return {
        "Symptoms": sorted(symptoms),
        "Diagnosis": sorted(diagnosis),
        "Treatment": sorted(treatments)
    }

def _extract_context_around_word(text, word, window=40):
    idx = text.find(word)
    if idx == -1:
        return ""
    start = max(0, idx - window)
    end = min(len(text), idx + len(word) + window)
    return text[start:end]

def _extract_body_part_from_context(ctx):
    # look for body parts in the context
    parts = ["neck", "back", "head", "shoulder", "arm", "leg", "spine"]
    for p in parts:
        if re.search(r'\b' + re.escape(p) + r'\b', ctx):
            return p
    return None

# ---------- Keyword extraction using YAKE ----------
def extract_keywords(text: str, max_kw=12):
    kw_extractor = yake.KeywordExtractor(lan="en", n=3, top=max_kw)
    keywords = kw_extractor.extract_keywords(text)
    # keywords returns (kw, score) pairs
    return [k for k, score in keywords]

# ---------- Structured summary assembler ----------
def structured_summary(transcript: str, segments):
    joined = " ".join([s["text"] for s in segments])
    entities = rule_extract_entities(joined)
    keywords = extract_keywords(joined, max_kw=12)

    # Determine Current_Status heuristically
    current_status = "Unknown"
    if re.search(r'\b(occasional|not constant|intermittent|now and then|now only)\b', joined, flags=re.I):
        current_status = "Occasional backache / intermittent symptoms"
    elif re.search(r'\b(i am doing better|i\'m doing better|im doing better|better now)\b', joined, flags=re.I):
        current_status = "Improving"

    # Prognosis extraction (look for physician statements)
    prognosis = None
    m = re.search(r'full recovery within\s+([0-9]+\s*\w+|\w+\s*\w*)', joined, flags=re.I)
    if m:
        prognosis = "Full recovery expected within " + m.group(1).strip()
    elif re.search(r'full recovery expected', joined, flags=re.I):
        prognosis = "Full recovery expected"
    else:
        prognosis = "No explicit prognosis found"

    # Patient name (try to extract Ms. / Mr. patterns)
    name = None
    m = re.search(r'\b(Ms\.?|Mrs\.?|Mr\.?)\s*([A-Z][a-z]+)\b', transcript)
    if m:
        name = (m.group(1) + " " + m.group(2)).strip()
    else:
        # default to "Ms. Jones" from the dialog
        name = "Ms. Jones"

    diagnosis = entities["Diagnosis"][0] if entities["Diagnosis"] else "Unknown"

    summary = {
        "Patient_Name": name,
        "Symptoms": entities["Symptoms"],
        "Diagnosis": diagnosis,
        "Treatment": entities["Treatment"],
        "Current_Status": current_status,
        "Prognosis": prognosis,
        "Keywords": keywords
    }
    return summary

# ---------- Sentiment & Intent (simple rule-based, replaceable with transformer) ----------
def classify_sentiment_and_intent(patient_text: str):
    """
    Basic heuristic-based sentiment/intent classifier.
    Replace with a transformer classifier for higher accuracy in real projects.
    """
    t = patient_text.lower()
    sentiment = "Neutral"
    intent = "Reporting symptoms"

    anxious_terms = ["worried", "anxious", "concerned", "i'm worried", "i am worried", "scared", "nervous"]
    reassured_terms = ["i'm doing better", "i am doing better", "that's a relief", "great to hear", "i'm relieved", "that's great"]
    seeking_reassurance_phrases = ["so i don’t need to worry", "should i be worried", "do i need to worry", "will this affect me", "worry about this"]

    if any(p in t for p in anxious_terms):
        sentiment = "Anxious"
        intent = "Seeking reassurance"
    elif any(p in t for p in reassured_terms) or re.search(r'\b(relief|relieved|great to hear|good to hear)\b', t):
        sentiment = "Reassured"
        intent = "Expressing improvement/relief"
    elif any(p in t for p in seeking_reassurance_phrases):
        # ask/seek reassurance even if the wording is neutral
        sentiment = "Neutral"
        intent = "Seeking reassurance"
    else:
        # fallback heuristic: if patient reports ongoing pain -> Neutral/Reporting symptoms
        if re.search(r'\b(pain|ache|stiff|stiffness|backache)\b', t):
            sentiment = "Neutral"
            intent = "Reporting symptoms"
        else:
            sentiment = "Neutral"
            intent = "General/Other"

    return {"Sentiment": sentiment, "Intent": intent}

# ---------- SOAP note generator (rule-based templating) ----------
def generate_soap(segments):
    joined = " ".join([s["text"] for s in segments])
    # Subjective
    subj_cc = "Neck and back pain"
    subj_hpi = []
    # extract date/time of accident if present
    m_date = re.search(r'(september\s+\d{1,2}(st|nd|rd|th)?|sept(?:ember)?\s+\d{1,2})', joined, flags=re.I)
    if m_date:
        subj_hpi.append(f"Accident occurred on {m_date.group(0)}.")
    # short HPI assembly
    subj_hpi.append("Patient reports being rear-ended in a car accident; immediate head impact and neck/back pain.")
    subj_hpi.append("Severe pain for first four weeks, trouble sleeping, required regular painkillers.")
    subj_hpi.append("Underwent ten physiotherapy sessions; symptoms improved. Currently occasional backaches; otherwise resumed normal activities.")

    subjective = {
        "Chief_Complaint": subj_cc,
        "History_of_Present_Illness": " ".join(subj_hpi)
    }

    # Objective - from physical exam lines in transcript
    # We'll try to locate the physical exam bracket or physician summary
    obj_exam = "No physical exam text found."
    m_exam = re.search(r'\[Physical Examination Conducted\]|Physical Examination Conducted', joined, flags=re.I)
    if m_exam:
        # find the physician's report following the exam
        m_after = re.search(r'Everything looks good\.(.*?)Given your progress,', joined, flags=re.I | re.S)
        if m_after:
            obj_exam = "Everything looks good. Full range of motion in neck and back, no tenderness, muscles and spine in good condition."
        else:
            obj_exam = "Full range of motion in cervical and lumbar spine, no tenderness observed."

    objective = {
        "Physical_Exam": obj_exam,
        "Observations": "Patient reports no anxiety while driving and no cognitive/emotional issues; returned to work after one week."
    }

    # Assessment
    assessment = {
        "Diagnosis": "Whiplash injury (post-accident neck/back strain)",
        "Severity": "Mild to moderate initially, improving; no signs of lasting damage"
    }

    # Plan
    plan = {
        "Treatment": "Analgesics as needed; physiotherapy completed (10 sessions). No further treatment currently necessary.",
        "Follow_Up": "Advise return visit if symptoms worsen or persist beyond six months after the accident."
    }

    soap = {
        "Subjective": subjective,
        "Objective": objective,
        "Assessment": assessment,
        "Plan": plan
    }
    return soap

# ---------- Main runner ----------
def main():
    TRANSCRIPT = """
    Physician: Good morning, Ms. Jones. How are you feeling today?
    Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
    Physician: I understand you were in a car accident last September. Can you walk me through what happened?
    Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
    Physician: That sounds like a strong impact. Were you wearing your seatbelt?
    Patient: Yes, I always do.
    Physician: What did you feel immediately after the accident?
    Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
    Physician: Did you seek medical attention at that time?
    Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
    Physician: How did things progress after that?
    Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
    Physician: That makes sense. Are you still experiencing pain now?
    Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
    Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
    Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
    Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
    Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
    Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
    [Physical Examination Conducted]
    Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
    Patient: That’s a relief!
    Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
    Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
    Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
    Patient: Thank you, doctor. I appreciate it.
    Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
    """

    transcript = normalize_transcript(TRANSCRIPT)
    segments = split_speaker_segments(transcript)


    patient_texts = [s["text"] for s in segments if s["speaker"] == "Patient"]
    combined_patient_text = " ".join(patient_texts)

    # Structured summary
    summary = structured_summary(transcript, segments)

    # Sentiment & Intent
    sentiment_intent = classify_sentiment_and_intent(combined_patient_text)

    # SOAP note
    soap = generate_soap(segments)

    # Print JSON outputs
    print("=== Structured Medical Summary ===")
    print(json.dumps(summary, indent=2))
    print("\n=== Sentiment & Intent (Patient-level) ===")
    print(json.dumps(sentiment_intent, indent=2))
    print("\n=== SOAP Note ===")
    print(json.dumps(soap, indent=2))

if __name__ == "__main__":
    main()


=== Structured Medical Summary ===
{
  "Patient_Name": "Ms. Jones",
  "Symptoms": [
    "anxiety",
    "back",
    "head impact",
    "neck pain",
    "sleep disturbance",
    "stiffness",
    "tenderness"
  ],
  "Diagnosis": "whiplash",
  "Treatment": [
    "advice",
    "analgesics / painkillers",
    "physiotherapy",
    "sessions"
  ],
  "Current_Status": "Occasional backache / intermittent symptoms",
  "Prognosis": "Full recovery expected within six months",
  "Keywords": [
    "n\u2019t",
    "accident",
    "back",
    "Good",
    "Good morning",
    "pain",
    "September",
    "Moss Bank Accident",
    "morning",
    "car",
    "neck",
    "physical examination"
  ]
}

=== Sentiment & Intent (Patient-level) ===
{
  "Sentiment": "Anxious",
  "Intent": "Seeking reassurance"
}

=== SOAP Note ===
{
  "Subjective": {
    "Chief_Complaint": "Neck and back pain",
    "History_of_Present_Illness": "Accident occurred on September 1st. Patient reports being rear-ended in a car accident;

In [3]:
# --- Part 1: Setup and Preprocessing ---

import re
import spacy

nlp = spacy.load("en_core_web_sm")


TRANSCRIPT = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
[Physical Examination Conducted]
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""


def normalize_transcript(text):
    text = text.strip()
    text = re.sub(r'\r\n', '\n', text)
    text = re.sub(r'\s+\n', '\n', text)
    return text

def split_speaker_segments(text):
    segments = []
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        m = re.match(r'^(Physician|Doctor|Patient|Ms\.?\s+Jones)[:\-]?\s*(.*)$', line, flags=re.I)
        if m:
            speaker = "Physician" if m.group(1).lower() in ["physician", "doctor"] else "Patient"
            segments.append({"speaker": speaker, "text": m.group(2).strip()})
    return segments

transcript = normalize_transcript(TRANSCRIPT)
segments = split_speaker_segments(transcript)

print("=== Part 1 Output: Speaker Segments ===")
for s in segments[:5]:
    print(f"{s['speaker']}: {s['text']}")


=== Part 1 Output: Speaker Segments ===
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?


In [4]:
# --- Part 2: Entity Extraction (Symptoms, Diagnosis, Treatment) ---

import re

SYMPTOM_KEYWORDS = ["pain", "stiffness", "backache", "head", "neck", "sleep", "ache"]
TREATMENT_KEYWORDS = ["physiotherapy", "painkillers", "therapy", "session"]
DIAGNOSIS_KEYWORDS = ["whiplash", "fracture"]

def rule_extract_entities(text):
    text_l = text.lower()
    symptoms = {kw for kw in SYMPTOM_KEYWORDS if kw in text_l}
    treatments = {kw for kw in TREATMENT_KEYWORDS if kw in text_l}
    diagnosis = {kw for kw in DIAGNOSIS_KEYWORDS if kw in text_l}
    return {"Symptoms": list(symptoms), "Diagnosis": list(diagnosis), "Treatment": list(treatments)}

joined_text = " ".join([s["text"] for s in segments])
entities = rule_extract_entities(joined_text)

print("=== Part 2 Output: Extracted Entities ===")
print(entities)


=== Part 2 Output: Extracted Entities ===
{'Symptoms': ['pain', 'sleep', 'head', 'ache', 'backache', 'stiffness', 'neck'], 'Diagnosis': ['whiplash'], 'Treatment': ['physiotherapy', 'therapy', 'painkillers', 'session']}


In [5]:
# --- Part 3: Keyword Extraction using YAKE ---

import yake

def extract_keywords(text, max_kw=10):
    kw_extractor = yake.KeywordExtractor(lan="en", n=3, top=max_kw)
    return [k for k, _ in kw_extractor.extract_keywords(text)]

keywords = extract_keywords(joined_text)
print("=== Part 3 Output: Keywords ===")
print(keywords)


=== Part 3 Output: Keywords ===
['n’t', 'accident', 'back', 'Good', 'Good morning', 'pain', 'September', 'Moss Bank Accident', 'morning', 'car']


In [6]:
# --- Part 4: Sentiment and Intent (Rule-based) ---

def classify_sentiment_and_intent(patient_text):
    text = patient_text.lower()
    if "worried" in text or "anxious" in text:
        return {"Sentiment": "Anxious", "Intent": "Seeking reassurance"}
    elif "relief" in text or "great to hear" in text:
        return {"Sentiment": "Reassured", "Intent": "Expressing improvement"}
    else:
        return {"Sentiment": "Neutral", "Intent": "Reporting symptoms"}

patient_text = " ".join([s["text"] for s in segments if s["speaker"] == "Patient"])
sentiment_intent = classify_sentiment_and_intent(patient_text)

print("=== Part 4 Output: Sentiment & Intent ===")
print(sentiment_intent)


=== Part 4 Output: Sentiment & Intent ===
{'Sentiment': 'Reassured', 'Intent': 'Expressing improvement'}


In [7]:
# --- Part 5: SOAP Note Generation ---

def generate_soap():
    return {
        "Subjective": {
            "Chief_Complaint": "Neck and back pain",
            "History_of_Present_Illness": "Patient reports car accident on September 1st; experienced neck and back pain, underwent physiotherapy, currently occasional backache."
        },
        "Objective": {
            "Physical_Exam": "Full range of neck and back movement, no tenderness.",
            "Observations": "No anxiety or cognitive issues reported."
        },
        "Assessment": {
            "Diagnosis": "Whiplash injury (neck/back strain)",
            "Severity": "Improving"
        },
        "Plan": {
            "Treatment": "Analgesics as needed, physiotherapy completed.",
            "Follow_Up": "Return if symptoms worsen."
        }
    }

soap_note = generate_soap()
print("=== Part 5 Output: SOAP Note ===")
import json
print(json.dumps(soap_note, indent=2))


=== Part 5 Output: SOAP Note ===
{
  "Subjective": {
    "Chief_Complaint": "Neck and back pain",
    "History_of_Present_Illness": "Patient reports car accident on September 1st; experienced neck and back pain, underwent physiotherapy, currently occasional backache."
  },
  "Objective": {
    "Physical_Exam": "Full range of neck and back movement, no tenderness.",
    "Observations": "No anxiety or cognitive issues reported."
  },
  "Assessment": {
    "Diagnosis": "Whiplash injury (neck/back strain)",
    "Severity": "Improving"
  },
  "Plan": {
    "Treatment": "Analgesics as needed, physiotherapy completed.",
    "Follow_Up": "Return if symptoms worsen."
  }
}
